Importantly, a convnet takes as input tensors of shape (image_height, image_width,
image_channels), not including the batch dimension. In this case, we’ll configure the
convnet to process inputs of size (28, 28, 1), which is the format of MNIST images.

the first convolution layer takes a feature map of size (28,
28, 1) and outputs a feature map of size (26, 26, 32): it computes 32 filters over its
input. Each of these 32 output channels contains a 26 × 26 grid of values, which is a
response map of the filter over the input, indicating the response of that filter pattern at
different locations in the input

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 128)         73856 

You can see that the output of every Conv2D and MaxPooling2D layer is a rank-3 tensor
of shape (height, width, channels). The width and height dimensions tend to
shrink as you go deeper in the model. The number of channels is controlled by the
first argument passed to the Conv2D layers (32, 64, or 128).

After the last Conv2D layer, we end up with an output of shape (3, 3, 128)—a 3 × 3
feature map of 128 channels. The next step is to feed this output into a densely connected classifier like those you’re already familiar with: a stack of Dense layers.

These
classifiers process vectors, which are 1D, whereas the current output is a rank-3 tensor.
To bridge the gap, we flatten the 3D outputs to 1D with a **Flatten layer** before adding
the Dense layers.

In [ ]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255
model.compile(optimizer="rmsprop",
 loss="sparse_categorical_crossentropy",
 metrics=["accuracy"])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/5
938/938 [==============================] - 52s 54ms/step - loss: 0.1651 - accuracy: 0.9488
Epoch 2/5
938/938 [==============================] - 52s 56ms/step - loss: 0.0457 - accuracy: 0.9858
Epoch 3/5
938/938 [==============================] - 54s 58ms/step - loss: 0.0311 - accuracy: 0.9901
Epoch 4/5
938/938 [==============================] - 50s 54ms/step - loss: 0.0241 - accuracy: 0.9928
Epoch 5/5
938/938 [==============================] - 53s 57ms/step - loss: 0.0182 - accuracy: 0.9946


In [ ]:
 test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 3s 10ms/step - loss: 0.0274 - accuracy: 0.9920


In [ ]:
print(f"Test accuracy: {test_acc:.3f}")

Test accuracy: 0.992


[How convolution works](https://chat.openai.com/share/9215d7d9-82b1-4531-9ca8-7be3e47583a5)

[Border effects and strides ](https://chat.openai.com/share/d7c92fb0-decc-43a0-8dd2-211014520a29)

[Max Pooling and Why we use it ](https://chat.openai.com/share/94447032-112e-4aaa-bf15-989c5cc399a8)